In [9]:
import sys
import numpy

# Verifica la versione di Python
print(f"Versione di Python: {sys.version}")

# Verifica la versione di NumPy
print(f"Versione di NumPy: {numpy.__version__}")

Versione di Python: 3.11.5 (main, Sep 11 2023, 13:54:46) [GCC 11.2.0]
Versione di NumPy: 2.1.3


In [1]:
import numpy as np
import itertools

def initialize_parameters(N, K, seed=None):
    #Initialize parameters randomly

    if seed is not None:
        np.random.seed(seed)

    pi = np.random.dirichlet(alpha=np.ones(K))    

    check = np.allclose(sum(pi), 1, atol=1e-6)
    if check != 1:
        print("Inconsistent boundary conditions")


    # Transition matrix. it has K rows and K columns. In particular
    # the sum of probabilities from the same initial state should be 1
    gamma = np.random.dirichlet(alpha=np.ones(K), size=K).T

    sum_columns = np.sum(gamma, axis = 0)
    check = np.allclose(sum_columns, np.ones(K), atol=1e-6)
    if check != 1:
        print("Inconsistent boundary conditions")


    # I initialize the r matrix in this in order to satisfy the boundary condition (sum of 
    # columns must be 1 foreach column)
    r = np.random.dirichlet(alpha=np.ones(N + 1), size=K).T

    sum_columns = np.sum(r, axis = 0)
    check = np.allclose(sum_columns, np.ones(K), atol=1e-6)
    if check != 1:
        print("Inconsistent boundary conditions")

    return pi, gamma, r

def generate_hmm_data(T, N, K, pi, gamma, r):
    z = np.zeros(T, dtype=int)  
    y = np.zeros(T, dtype=int)  

    # Initialization t=0. I extract the value of K following the init prob
    z[0] = np.random.choice(K, p=pi)

    #data generation
    for t in range(T):
        if t > 0:
            # I take the column of gamma corresponding to z[t-1] and extract from this distribution
            z[t] = np.random.choice(K, p=gamma[:, z[t-1]])

        # I take the column of r corresponding to z[t] and extract from this prob distribution
        y[t] = np.random.choice(N + 1, p=r[:, z[t]])

    return y, z

def likelihood_comp(T, K, pi, gamma, r, y):

    likelihood = 0
    # all possible sequences z
    all_sequences = itertools.product(range(K), repeat=T)

    for z in all_sequences: 

        p_1 = 0
        p_2 = pi[z[0]]

        for t in range(T-1):
            p_1 *= r[y[t], z[t]]
            p_2 *= gamma[z[t], z[t+1]]

        p_1 *= r[y[T-1], z[T-1]]

        likelihood += (p_1*p_2)

    return likelihood

    


N = 5  # neurons
K = 3  # states
T = 500  # time steps
seed = None #1234  

#input: initialization of the parameters for the HMM
pi, gamma, r = initialize_parameters(N, K, seed)

print("pi = ", pi)
print("gamma = ", gamma)
print("r = ", r)

# print(r[y[0]])

#output: generation of the observed and hidden data
y, z = generate_hmm_data(T, N, K, pi, gamma, r)

print("y:", y)
print("z:", z)

# likelihood = likelihood_comp(T, K, pi, gamma, r, y)
# print(likelihood)

pi =  [0.19859089 0.28084176 0.52056736]
gamma =  [[0.19304856 0.31028103 0.67567619]
 [0.27708523 0.67070171 0.0603891 ]
 [0.52986621 0.01901726 0.26393471]]
r =  [[0.25876785 0.05328365 0.03235647]
 [0.02644361 0.07661813 0.05726404]
 [0.07944354 0.23170179 0.00214624]
 [0.35272302 0.15920893 0.39489557]
 [0.11068831 0.34219423 0.06721013]
 [0.17193367 0.13699327 0.44612755]]
y: [5 1 3 4 4 4 5 2 3 0 0 4 4 0 3 4 3 5 4 5 0 3 4 5 3 3 3 4 1 4 4 3 5 0 3 5 4
 0 3 5 5 4 3 3 5 3 3 0 2 3 5 5 0 3 5 2 5 4 3 0 2 0 3 0 3 5 3 5 3 0 2 2 4 3
 3 1 3 4 3 1 0 3 3 5 3 4 2 2 4 0 5 3 5 4 2 5 5 3 3 1 2 5 2 5 2 0 2 5 1 3 3
 0 3 5 1 5 5 3 5 5 3 0 5 4 3 3 4 3 3 5 0 4 5 3 3 5 1 1 5 3 3 2 4 4 3 2 0 0
 4 3 5 4 2 5 3 4 3 5 5 4 3 3 0 3 3 1 5 3 3 5 5 4 3 0 3 3 3 3 5 5 3 3 5 5 3
 0 4 2 2 5 5 4 4 0 3 4 4 3 2 4 2 5 5 5 0 0 5 5 2 5 0 2 5 3 0 5 3 3 3 3 4 0
 5 0 4 5 2 3 3 5 3 3 4 1 2 2 3 0 3 2 1 3 3 5 0 0 5 4 4 3 5 3 3 5 3 0 3 3 3
 0 3 0 0 1 3 5 3 4 3 4 1 5 1 5 5 0 3 2 3 3 4 5 5 0 3 0 3 4 0 5 4 5 4 4 5 4
 3 5 3 5 5 3 5 3

In [3]:
import time
from tqdm import tqdm

def forward(y, pi, gamma, r, T, K, epsilon=1e-10):
    
    alpha = np.zeros((T, K))   # alpha has T elements of dimension K each

    alpha[0] = pi * r[y[0]]   # r[y[0]] is the first row of r: r_y1,k  

    for t in range(1, T):   
        for k in range(K):
            alpha[t, k] = r[y[t-1], k] * np.sum(alpha[t - 1] * gamma[k]) + epsilon     #gamma[k] is the k'th row of gamma

    """
    print(alpha)
    print(pi)
    print(r[y[0]])
    """
    
    return alpha

def backward(y, gamma, r, T, K, epsilon=1e-10):
    
    beta = np.zeros((T, K))
    beta[-1] = 1

    for t in range(T - 2, -1, -1):
        for k in range(K):
            beta[t, k] = r[y[t+1], k] * np.sum(beta[t + 1] * gamma[k]) + epsilon

    #print(beta)

    return beta

def e_step(y, pi, gamma, r, T, K):
    
    alpha = forward(y, pi, gamma, r, T, K)
    beta = backward(y, gamma, r, T, K)
    #print("alpha = ", alpha)
    #print("beta = ", beta)

    # it must have T-1 elements because it's the P(z_t = k and z_t+1 = l | ...)
    xi = np.zeros((T - 1, K, K))
    for t in range(T - 1):
        for k in range(K):
            for l in range(K):
                xi[t, k, l] = alpha[t, k] * gamma[k, l] * r[y[t + 1], l] * beta[t + 1, l]
        #renormalization
        xi[t] /= np.sum(xi[t])

    zeta = np.zeros((T, K))
    for t in range(0, T):
        for k in range(K):
            zeta[t,k] = alpha[t, k] * beta[t, k]

        zeta[t] /= np.sum(zeta[t])

    """
    print("zeta = ", zeta)
    print("xi = ", xi)
    """

    return zeta, xi


def m_step(y, zeta, xi, N, K):
    
    T = len(y)
    # update of pi
    pi = np.zeros(K)
    pi = zeta[0]    
    #print(pi)

    # update of gamma
    gamma = np.zeros((K,K))
    for k in range(K):
        denominator = np.sum(zeta[:, k])
        for l in range(K):
            numerator = np.sum(xi[:, k, l])
            gamma[k,l] = numerator/denominator 

    # update of r 
    r = np.zeros((N + 1, K))

    for k in range(K):
        denominator = 0
        for i in range(N + 1):
            numerator = 0
            for t in range(T):
                if y[t]==i:
                    numerator += zeta[t, k]
                    denominator += zeta[t,k]
        
            r[i,k] = numerator
        
        r[:, k] /= denominator

    return pi, gamma, r

def em_algorithm(y, N, K, max_iter=100, tol=1e-6):
    
    T = len(y)
    pi, gamma, r = initialize_parameters(N, K)

    print("Initialized pi: ", pi)
    print("Initialized gamma: ", gamma)
    print("Initialized r: ", r)

    for iteration in tqdm(range(max_iter),desc=f"Running EM algorithm with {max_iter} iterations"):
        #print(iteration)
        # E-step
        zeta, xi = e_step(y, pi, gamma, r, T, K)

        #print("zeta = ", zeta)
        #print("xi = ", xi)
        time.sleep(1)

        # M-step
        pi_updated, gamma_updated, r_updated = m_step(y, zeta, xi, N, K)

        """
        print("pi_upd = ", pi_updated)
        print("gamma_upd = ", gamma_updated)
        print("r_upd = ", r_updated)
        """
    
        # i compute the delta using the relative distance, and using the Frobenius norm
        # delta_pi = np.linalg.norm(pi_updated - pi, ord='fro') / (np.linalg.norm(pi, ord='fro') + 1e-10)
        delta_pi = np.linalg.norm(pi_updated - pi)
        delta_gamma = np.linalg.norm(gamma_updated - gamma, ord='fro')
        delta_r = np.linalg.norm(r_updated - r, ord='fro')
        

        # if delta_pi < tol and delta_gamma < tol and delta_r < tol:
        if delta_gamma < tol and delta_r < tol and delta_pi < tol:
            print(f"Converged at iteration {iteration}")
            break

        pi = pi_updated
        gamma = gamma_updated
        r = r_updated

    return pi_updated, gamma_updated, r_updated

pi_est, gamma_est, r_est = em_algorithm(y, N, K, max_iter=300)

print("pi_est = ", pi_est)
print("gamma_est = ", gamma_est)
print("r_est = ", r_est)

print("pi = ", pi)
print("gamma = ", gamma)
print("r = ", r)


Initialized pi:  [0.13897077 0.70232521 0.15870402]
Initialized gamma:  [[0.23665837 0.42875368 0.03112315]
 [0.16943419 0.19384707 0.33168114]
 [0.59390744 0.37739925 0.63719571]]
Initialized r:  [[0.02826684 0.11779465 0.22476547]
 [0.10982478 0.14801405 0.19239214]
 [0.77316949 0.57436303 0.04179336]
 [0.00554821 0.01902897 0.06547029]
 [0.07910432 0.05724951 0.156338  ]
 [0.00408635 0.08354979 0.31924074]]


Running EM algorithm with 1000 iterations: 100%|██████████| 1000/1000 [17:30<00:00,  1.05s/it]

pi_est =  [3.22651986e-11 3.46837808e-08 9.99999965e-01]
gamma_est =  [[0.10100883 0.68612583 0.21037866]
 [0.02777344 0.11964018 0.85115798]
 [0.04296231 0.12284583 0.83210645]]
r_est =  [[0.13646879 0.13618067 0.13535528]
 [0.04682201 0.04873124 0.04844175]
 [0.11238107 0.11212873 0.11149395]
 [0.32091521 0.32043651 0.31865802]
 [0.16662988 0.16624085 0.16513563]
 [0.21678305 0.216282   0.22091539]]
pi =  [0.19859089 0.28084176 0.52056736]
gamma =  [[0.19304856 0.31028103 0.67567619]
 [0.27708523 0.67070171 0.0603891 ]
 [0.52986621 0.01901726 0.26393471]]
r =  [[0.25876785 0.05328365 0.03235647]
 [0.02644361 0.07661813 0.05726404]
 [0.07944354 0.23170179 0.00214624]
 [0.35272302 0.15920893 0.39489557]
 [0.11068831 0.34219423 0.06721013]
 [0.17193367 0.13699327 0.44612755]]


In [ ]:
import numpy as np
from scipy.special import logsumexp
import time

def forward_log(y, pi, gamma, r, T, K):
    # Log of small probabilities start as -inf
    log_alpha = np.full((T, K), -np.inf)  
    log_alpha[0] = np.log(pi) + np.log(r[y[0]])  # Initialization

    for t in range(1, T):   
        for k in range(K):
            # Logarithmic forward step to avoid underflow
            log_alpha[t, k] = np.log(r[y[t-1], k]) + logsumexp(log_alpha[t - 1] + np.log(gamma[k]))
    
    return log_alpha

def backward_log(y, gamma, r, T, K):
    # Logarithmic backward step
    log_beta = np.full((T, K), -np.inf)
    log_beta[-1] = 0  # Log(1) = 0 for the final timestep

    for t in range(T - 2, -1, -1):
        for k in range(K):
            log_beta[t, k] = logsumexp(np.log(gamma[k]) + np.log(r[y[t+1]]) + log_beta[t + 1])
    
    return log_beta

def e_step_log(y, pi, gamma, r, T, K):
    log_alpha = forward_log(y, pi, gamma, r, T, K)
    log_beta = backward_log(y, gamma, r, T, K)

    log_likelihood = logsumexp(log_alpha[-1])  # Log of total probability

    # Zeta (using logsumexp to avoid underflow)
    log_zeta = log_alpha + log_beta - log_likelihood
    zeta = np.exp(log_zeta)

    # Xi (using logsumexp to avoid underflow)
    log_xi = np.zeros((T - 1, K, K))
    for t in range(T - 1):
        for k in range(K):
            for l in range(K):
                log_xi[t, k, l] = log_alpha[t, k] + np.log(gamma[k, l]) + np.log(r[y[t + 1], l]) + log_beta[t + 1, l] - log_likelihood
        # Renormalization (logspace sum)
        log_xi[t] -= logsumexp(log_xi[t])  # This normalizes log_xi[t] to prevent large values

    xi = np.exp(log_xi)

    return zeta, xi

def m_step(y, zeta, xi, N, K):
    T = len(y)
    
    # Update of pi
    pi = zeta[0]
    pi /= np.sum(pi)  # Normalize pi to ensure it's a valid probability distribution
    
    # Update of gamma
    gamma = np.zeros((K, K))
    for k in range(K):
        denominator = np.sum(zeta[:, k])
        for l in range(K):
            numerator = np.sum(xi[:, k, l])
            gamma[k, l] = numerator / denominator

    # Update of r
    r = np.zeros((N + 1, K))

    for k in range(K):
        denominator = 0
        for i in range(N + 1):
            numerator = 0
            for t in range(T):
                if y[t] == i:
                    numerator += zeta[t, k]
                    denominator += zeta[t, k]
        
            r[i, k] = numerator

        r[:, k] /= denominator

    return pi, gamma, r

def em_algorithm(y, N, K, max_iter=100, tol=1e-6):
    T = len(y)
    pi, gamma, r = initialize_parameters(N, K)

    print("Initialized pi: ", pi)
    print("Initialized gamma: ", gamma)
    print("Initialized r: ", r)

    for iteration in range(max_iter):
        # E-step
        zeta, xi = e_step_log(y, pi, gamma, r, T, K)

        print("zeta = ", zeta)
        print("xi = ", xi)

        # M-step
        pi_updated, gamma_updated, r_updated = m_step(y, zeta, xi, N, K)
    
        # Check for convergence using Frobenius norm or another metric
        delta_pi = np.linalg.norm(pi_updated - pi)
        delta_gamma = np.linalg.norm(gamma_updated - gamma, ord='fro')
        delta_r = np.linalg.norm(r_updated - r, ord='fro')

        if delta_gamma < tol and delta_r < tol and delta_pi < tol:
            print(f"Converged at iteration {iteration}")
            break

        pi = pi_updated
        gamma = gamma_updated
        r = r_updated

    return pi_updated, gamma_updated, r_updated

# Example usage
pi_est, gamma_est, r_est = em_algorithm(y, N, K, max_iter=1000)

print("Real pi: ", pi)
print("Real gamma: ", gamma)
print("Real r: ", r)

print("Estimated pi: ", pi_est)
print("Estimated gamma: ", gamma_est)
print("Estimated r: ", r_est)
